# Piotr Styczyński WBO - Zadanie 2 (Duże)

## Raport

Poniższy raport został porządzony w formacie Markdown jako notebook pythonowy w celu ułatwienia wizualnych prezentacji danych.

## Rozwiązania i struktura kodu

Pliki:
* extend.py
* fisher.py
* scan_pfam.py

Odpowiadają za odpowiednie podpunkty zadania. Skrypty używają biblioteki click w celu udostępnienia przyjemnego interfejsu CLI. Rozwiazania wykorzystują shared thread pool aby fetchowanie i obróbka danych była możliwie "jak najbardziej" asynchroniczna. Skrypty wspierają podanie wielu plików do przetworzenia a także pobieranie linków:

In [9]:
# Można uruchomić skrypt stąd:
!python extend.py "http://regulomics.mimuw.edu.pl/wp/wp-content/uploads/2019/05/input-z2.fasta_.txt"

Usage: extend.py [OPTIONS] [INPUT]...

Options:
  -l, --loggingLevel [INFO|DEBUG]
                                  Set logging level
  -t, --tmp TEXT                  Path to save temporary data. Default is
                                  ./workdir
  -i, --idThreshold INTEGER       Set minimal identity threshold in percents.
                                  Default is 90
  -e, --eValue TEXT               Set minimal E-value using scientific
                                  notation. Default is 10e-10
  --help                          Show this message and exit.


## Wyniki

Omówienie wyników znajduje się poniżej.

In [3]:
# Zaimportuj plotly i ustaw API KEY
import plotly.tools
plotly.tools.set_credentials_file(username='styczynski', api_key='oSxsY0BtX3lFusIhgSy0')

In [5]:
# Przetwórz dane funkcjami udostępnianymi przez fisher.py
import fisher

import plotly as py
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

DEFAULT_FILE_TMP_FOLDER = "./workdir"
input = ['./data/ext_hmm.csv', './data/nor_hmm.csv']
tmp = DEFAULT_FILE_TMP_FOLDER

inputCount = len(input)
if inputCount != 2:
    print("Invalid files number was specified. Please provide exactly two input files.")
    exit(1)

inputA = input[0]
inputB = input[1]
print("Fisher test of {} compared to {}".format(inputA, inputB))

tableA = fisher.loadMatrixFromCSV(inputA)
tableB = fisher.loadMatrixFromCSV(inputB)   
(domainNames, tables) = fisher.normalizeTables([ tableA, tableB ])

domainPsCustom = [round(v, 2) for v in fisher.customFisher(domainNames, tables[0], tables[1])]
domainPsFisher = [round(v, 2) for v in fisher.scipyFisher(domainNames, tables[0], tables[1])]
stats = fisher.perDomainStats(domainNames, tables[0], tables[1])

domainNamesF = []
for name in domainNames:
    if name != '':
        domainNamesF.append(name)
domainNames = domainNamesF

results = [ [r[0], r[1], r[2]] for r in zip(domainNames, domainPsCustom, domainPsFisher) ]
results.sort(key=lambda x: x[1])

percAStats = []
percBStats = []
for s in stats:
    percAStats.append(s[0])
    percBStats.append(s[1])


Fisher test of ./data/ext_hmm.csv compared to ./data/nor_hmm.csv


### Test Fishera

Poniżej znajduje się tabela z wynikami.
Wyniki SciPy oraz funkcji podanej w opisie zadania pokrywają się, świadczy to więc o poprawności rozwiązania.

Jak widać z przedstawionych danych Helicase_C ma znaczący wynik (mniej - większa istotność) natomiast DUF4291, Cyt-b5 są mniej istotnymi wynikami.


In [6]:
table = ff.create_table([["Domain name", "Custom Fisher", "SciPy Fisher"]] + results)
py.iplot(table, filename='jupyter-table1')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~styczynski/0 or inside your plot.ly account where it is named 'jupyter-table1'


/home/students/inf/p/ps386038/jupyter/env/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



### Procentowa ilość trafień

Poniżej widać procentową ilość trafień domeny w stosunku do liczby wszystkich sekwencji.

Helicase_C występuje w niemalże wszystkich próbkach oryginalnych i w małej części rozszerzonych sekwencji, natomiast ResIII w obu przypadkach ma podobną czestość występowania.



In [7]:
data = [go.Bar(name="% of matches in original", x=domainNames, y=percAStats), go.Bar(name="% of matches in extended", x=domainNames, y=percBStats)]
py.iplot(data, filename='jupyter-basic_bar')